In [9]:
# scrape the missing housing price indices
import pandas as pd
import numpy as np
import time
import requests
import plotly.express as px
import warnings

In [5]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [10]:
def get_listings(api_key, listing_url):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"

    querystring = {
        "api_key": api_key,
        "url":listing_url
    }

    return requests.request("GET", url, params=querystring)

def get_property_detail(api_key, zpid):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/property"

    querystring = {
        "api_key": api_key,
        "zpid":zpid
    }

    return requests.request("GET", url, params=querystring)

def get_zpid(api_key, street, city, state, zip_code=None):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/zpidByAddress"

    querystring = {
        "api_key": api_key,
        "street": street,
        "city": city,
        "state": state,
        "zip_code":zip_code
    }

    return requests.request("GET", url, params=querystring)

In [17]:
target = ['ARMOUR SQUARE', 'AUBURN GERMAN', 'AUSTIN']

In [11]:
api_key = "5e319a14-cab1-4a67-8d62-1c7c915b6df5"

In [16]:
hash = {"Community Name" : [], "Price" : []}

In [18]:
url = "https://www.zillow.com/armour-square-chicago-il/sold/?searchQueryState=%7B%22isMapVisible%22%3Afalse%2C%22mapBounds%22%3A%7B%22north%22%3A41.860083%2C%22south%22%3A41.823853%2C%22east%22%3A-87.615752%2C%22west%22%3A-87.65198199999999%7D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22rs%22%3A%7B%22value%22%3Atrue%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A269569%2C%22regionType%22%3A8%7D%5D%2C%22pagination%22%3A%7B%7D%7D"
# get listings
listing_response = get_listings(api_key, url)

In [19]:
listing_response.json().keys()
listing_response.json()["is_success"]

True

In [20]:
# view count of properies returned in request
num_of_properties = listing_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

Count of properties: 2550


In [21]:
df_listings = pd.json_normalize(listing_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_listings))
print("Number of columns:", len(df_listings.columns))
df_listings

Number of rows: 500
Number of columns: 86


,zpid,rawHomeStatusCd,marketingStatusSimplifiedCd,imgSrc,hasImage,detailUrl,statusType,statusText,price,priceLabel,address,beds,baths,area,isUserClaimingOwner,isUserConfirmedClaim,pgapt,sgapt,shouldShowZestimateAsPrice,has3DModel,hasVideo,isHomeRec,hasAdditionalAttributions,isFeaturedListing,isShowcaseListing,listingType,isFavorite,visited,brokerName,timeOnZillow,latLong.latitude,latLong.longitude,variableData.type,variableData.text,hdpData.homeInfo.zpid,hdpData.homeInfo.streetAddress,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.dateSold,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.zestimate,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.timeOnZillow,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.isShowcaseListing,minBeds,minBaths,minArea,lotId,buildingId,isBuilding,canSaveBuilding,unitCount,hdpData.homeInfo.unit,plid,availabilityDate,hdpData.homeInfo.isRentalWithBasePrice,streetViewURL,streetViewMetadataURL,lotAreaString,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceReduction,hdpData.homeInfo.priceChange,hdpData.homeInfo.listing_sub_type.is_FSBA
0,3882364,RecentlySold,RecentChange,https://photos.zillowstatic.com/fp/c59d5d347f5...,True,/homedetails/3419-S-Indiana-Ave-Chicago-IL-606...,SOLD,Closed,"$495,000",$495K,"3419 S Indiana Ave, Chicago, IL 60616",3.0,2.0,1871.0,False,False,RecentlySold,RecentChange,False,False,False,False,True,False,False,,False,False,Compass,247893000,41.832275,-87.621430,RECENTLY_SOLD,Sold 04/19/2024,3882364.0,3419 S Indiana Ave,60616,Chicago,IL,41.832275,-87.62143,495000.0,1.713510e+12,2.0,3.0,1871.0,SINGLE_FAMILY,RECENTLY_SOLD,-1.0,False,False,492000.0,2673.0,False,False,RECENTLY_SOLD,495000.0,2.478930e+08,True,False,False,USD,USA,319990.0,3125.000,sqft,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,https://photos.zillowstatic.com/fp/9f6d53e924e...,True,/b/2016-s-wabash-ave-chicago-il-BJ88/,SOLD,For Rent,,NaN,"2024 S Wabash Ave, Chicago, IL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,False,False,False,False,,False,NaN,NaN,247893000,41.854515,-87.626030,3D_HOME,3D Tour,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,803.0,9.751060e+05,41.854515--87.62603,True,False,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,https://photos.zillowstatic.com/fp/9b80839c120...,True,/b/3500-s-sangamon-st-chicago-il-5XkKM6/,SOLD,For Rent,,NaN,"3500 S Sangamon St, Chicago, IL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,False,False,,False,NaN,NaN,247893000,41.830517,-87.649940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,714.0,1.001583e+09,41.830517--87.64994,True,False,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,https://photos.zillowstatic.com/fp/5e2032deb8b...,True,/b/1620-s-michigan-ave-chicago-il-CMWT/,SOLD,For Rent,,NaN,"1620 S Michigan Ave, Chicago, IL",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,False,False,False,False,,False,NaN,NaN,247893000,41.859566,-87.624270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,607.0,1.100906e+06,41.859566--87.62427,True,False,53.0,NaN,NaN,

In [24]:
df_listings['hdpData.homeInfo.price'].median()

455000.0